# Database Connection

In [1]:
from pymongo import MongoClient

#MongoDB Connection
client = MongoClient('mongodb://localhost:27017/')

# Create Database and Collection

In [2]:
# Create Database
db = client['VideoKeyframeDB']

# Create the Collection
collection = db['videos']

# Prepare and insert Data

In [4]:
import os
import cv2

def parse_shot_boundaries(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        shots = [line.strip().split(', ') for line in lines]
        return shots

def extract_framerate(video_path):
    video = cv2.VideoCapture(video_path)
    framerate = video.get(cv2.CAP_PROP_FPS)
    video.release()
    return framerate

def read_keyframe_image(file_path):
    with open(file_path, 'rb') as f:
        return f.read()

def prepare_video_data(video_id, collection_name, video_path, shot_boundaries_path, keyframes_path):
    original_framerate = extract_framerate(video_path)
    shots = parse_shot_boundaries(shot_boundaries_path)
    frames = []
    for i, (start, end) in enumerate(shots, start=1):
        frame_id = f"{video_id}_Scene-{i}"
        frame = {
            "FrameID": frame_id,
            "Starttime": start,
            "Endtime": end,
            "Objects": []
        }
        frames.append(frame)

    video_data = {
        "VideoID": video_id,
        "CollectionName": collection_name,
        "OriginalFramerate": original_framerate,
        "EditedFramerate": "Null",
        "Frames": frames
    }
    return video_data

def insert_all_videos(base_video_path, base_shot_boundaries_path, base_keyframes_path, collection_name):
    for video_num in range(100, 200): # from 00100 to 00199
        video_id = f"{video_num:05d}"
        video_path = os.path.join(base_video_path, f"{video_id}.mp4")
        shot_boundaries_path = os.path.join(base_shot_boundaries_path, f"{video_id}_shots.txt")
        keyframes_path = os.path.join(base_keyframes_path, video_id)

        if os.path.exists(shot_boundaries_path) and os.path.exists(keyframes_path):
            video_data = prepare_video_data(video_id, collection_name, video_path, shot_boundaries_path, keyframes_path)
            collection.insert_one(video_data)
        else:
            print(f"Missing Video Data for Video {video_id}: {shot_boundaries_path} or {keyframes_path}")

#Change according to config
base_video_path = 'V3C-100'
base_shot_boundaries_path = 'shot_boundaries'
base_keyframes_path = 'keyframes'
collection_name = 'V3C100'

insert_all_videos(base_video_path, base_shot_boundaries_path, base_keyframes_path, collection_name)